In [9]:
# CREATE TRANSIT  STOP FACILITIES

import pandas as pd
import xml.etree.ElementTree as ET
from xml.dom import minidom


# Liste der CSV-Dateien
csv_files = [r'C:\Users\Julius Guizetti\IdeaProjects\matsim-hamburg-HA2\scenarios\input\stationen_ba.csv', r'C:\Users\Julius Guizetti\IdeaProjects\matsim-hamburg-HA2\scenarios\input\stationen_ab.csv']

xml_file = r'C:\Users\Julius Guizetti\IdeaProjects\matsim-hamburg-HA2\scenarios\input\adapt_transit-schedule.xml'  # Neue XML-Datei, die erstellt wird

# Lese die CSV-Datei in ein Pandas DataFrame ein
df = pd.read_csv(csv_file, sep=';')


# Erstelle ein XML-Dokument für das gesamte transitSchedule
transit_schedule = ET.Element('transitSchedule')

# Iteriere über jede CSV-Datei
for csv_file in csv_files:
    # Lese die CSV-Datei in ein Pandas DataFrame ein
    # Da die Spalten durch Semikolon getrennt sind, verwenden wir den Parameter sep=';'
    df = pd.read_csv(csv_file, sep=';')
    
    # Erstelle ein transitStops-Element innerhalb von transitSchedule
    transit_stops = ET.SubElement(transit_schedule, 'transitStops')
    
    # Iteriere über jede Zeile im DataFrame
    for index, row in df.iterrows():
        # Erstelle ein neues Stopfacility-Element
        stop_facility = ET.SubElement(transit_stops, 'stopFacility')
        stop_facility.set('id', str(row['StopFacility']))
        stop_facility.set('x', str(row['x']))
        stop_facility.set('y', str(row['y']))
        stop_facility.set('linkRefId', str(row['LinkID']))
        stop_facility.set('isBlocking', 'false')

# Erstelle eine formatierte XML-Ausgabe für bessere Lesbarkeit
xml_str = ET.tostring(transit_schedule, encoding='utf-8')
xml_str = minidom.parseString(xml_str).toprettyxml(indent="  ")

# Speichere die XML-Datei
with open(xml_file, 'w') as f:
    f.write(xml_str)

print(f"Die XML-Datei '{xml_file}' wurde erfolgreich erstellt.")

Die XML-Datei 'C:\Users\Julius Guizetti\IdeaProjects\matsim-hamburg-HA2\scenarios\input\adapt_transit-schedule.xml' wurde erfolgreich erstellt.


In [13]:
# CREATE TRANSIT LINES AND TRANSIT ROUTES AND STOPS

import pandas as pd
import xml.etree.ElementTree as ET
from xml.dom import minidom

# Dateinamen der CSV-Dateien
csv_files = ['stationen_ba.csv', 'stationen_ab.csv']

# Hilfsfunktion zur Umwandlung von Sekunden in hh:mm:ss
#def seconds_to_hhmmss(seconds):
 #   hours = seconds // 3600
  #  minutes = (seconds % 3600) // 60
   # seconds = seconds % 60
    #return f"{hours:02}:{minutes:02}:{seconds:02}"

# Erstelle ein XML-Dokument für das gesamte transitSchedule
transit_schedule = ET.Element('transitSchedule')

# Erstelle eine transitLine mit der ID 'U5'
transit_line = ET.SubElement(transit_schedule, 'transitLine', id='U5')

# Funktion zum Hinzufügen von transitRoute
def add_transit_route(route_id, csv_file):
    # Lese die CSV-Datei in ein Pandas DataFrame ein
    df = pd.read_csv(csv_file, sep=';')
    
    # Erstelle eine transitRoute mit der angegebenen ID
    transit_route = ET.SubElement(transit_line, 'transitRoute', id=route_id)
    
    # Füge das transportMode-Element hinzu
    transport_mode = ET.SubElement(transit_route, 'transportMode')
    transport_mode.text = 'pt'
    
    # Füge das routeProfile-Element hinzu
    route_profile = ET.SubElement(transit_route, 'routeProfile')
    
    # Iteriere über jede Zeile im DataFrame und füge stop-Elemente hinzu
    for index, row in df.iterrows():
        stop = ET.SubElement(route_profile, 'stop', refId=str(row['StopFacility']))
        stop.set('arrivalOffset', str(row['arrivalOffset']))
        stop.set('departureOffset', str(row['departureOffset']))
        stop.set('awaitDeparture', 'true')

# Füge die beiden transitRoutes hinzu
add_transit_route('U5_BA', csv_files[0])
add_transit_route('U5_AB', csv_files[1])

# Erstelle eine formatierte XML-Ausgabe für bessere Lesbarkeit
xml_str = ET.tostring(transit_schedule, encoding='utf-8')
xml_str = minidom.parseString(xml_str).toprettyxml(indent="  ")

# Speichere die XML-Datei
xml_file = 'adapt_transit-schedule_offset_correction.xml'  # Neue XML-Datei, die erstellt wird
with open(xml_file, 'w') as f:
    f.write(xml_str)

print(f"Die XML-Datei '{xml_file}' wurde erfolgreich erstellt.")


Die XML-Datei 'adapt_transit-schedule_offset_correction.xml' wurde erfolgreich erstellt.


In [23]:
# CREATE ROUTES AS LIST OF LINKS
import xml.etree.ElementTree as ET
from xml.dom import minidom

# Dateiname der vorhandenen XML-Datei mit den Transitdaten
existing_xml_file = r'C:\Users\Julius Guizetti\IdeaProjects\matsim-hamburg-HA2\scenarios\input\adapt_transit-schedule_lines.xml'

# Lese die vorhandene XML-Datei ein
tree_existing = ET.parse(existing_xml_file)
root_existing = tree_existing.getroot()

# Dateiname der XML-Datei mit den Links für U5_AB und U5_BA
links_xml_file = r'C:\Users\Julius Guizetti\IdeaProjects\matsim-hamburg-HA2\scenarios\input\adapt_network.xml'

# Lese die XML-Datei mit den Links ein
tree_links = ET.parse(links_xml_file)
root_links = tree_links.getroot()

# Funktion zum Einfügen der Links in die TransitRouten
def insert_links_into_route(route_id, links):
    # Suche die TransitRoute mit der gegebenen ID in der vorhandenen XML-Datei
    transit_routes = root_existing.findall(f".//transitRoute[@id='{route_id}']")
    if len(transit_routes) != 1:
        raise ValueError(f"TransitRoute mit ID '{route_id}' wurde nicht gefunden oder ist nicht eindeutig.")
    
    transit_route = transit_routes[0]

    # Erstelle das transitRoute-Element, falls es noch nicht existiert
    route_element = ET.SubElement(transit_route, 'route')

    # Füge die Links in der richtigen Reihenfolge hinzu
    for link in links:
        link_element = ET.SubElement(route_element, 'link')
        link_element.set('refId', link.attrib['id'])  # Setze die refId des Links

# Extrahiere und füge die Links für U5_AB und U5_BA ein
for route_id in ['U5_AB', 'U5_BA']:
    # Filtere Links für die aktuelle Route
    if route_id == 'U5_AB':
        links_for_route = [link for link in root_links.findall('.//link') if link.attrib['id'].startswith("AB-")]
    elif route_id == 'U5_BA':
        links_for_route = [link for link in root_links.findall('.//link') if link.attrib['id'].startswith("BA-")]
    else:
        continue

    # Füge die Links in die TransitRoute ein
    insert_links_into_route(route_id, links_for_route)

## Funktion zum Formatieren des XML-Dokuments
def format_xml(element, level=0):
    indent = "    "
    if len(element):
        if not element.text or not element.text.strip():
            element.text = "\n" + indent * (level + 1)
        if not element.tail or not element.tail.strip():
            element.tail = "\n" + indent * level
        for elem in element:
            format_xml(elem, level + 1)
        if not element.tail or not element.tail.strip():
            element.tail = "\n" + indent * level
    else:
        if level and (not element.tail or not element.tail.strip()):
            element.tail = "\n" + indent * level

# XML-Dokument formatieren
format_xml(root_existing)

# Speichere die aktualisierte XML-Datei
xml_str = ET.tostring(root_existing, encoding='utf-8')
xml_str = minidom.parseString(xml_str).toprettyxml(indent="  ")

with open(existing_xml_file, 'w') as f:
    f.write(xml_str)

print(f"Die XML-Datei '{existing_xml_file}' wurde erfolgreich aktualisiert.")

Die XML-Datei 'C:\Users\Julius Guizetti\IdeaProjects\matsim-hamburg-HA2\scenarios\input\adapt_transit-schedule_lines.xml' wurde erfolgreich aktualisiert.


In [24]:
#Hilfe für formatierung
import xml.etree.ElementTree as ET

# Dateiname der vorhandenen XML-Datei
existing_xml_file = r'C:\Users\Julius Guizetti\IdeaProjects\matsim-hamburg-HA2\scenarios\input\adapt_transit-schedule_lines.xml'

# Lese die vorhandene XML-Datei ein
tree = ET.parse(existing_xml_file)
root = tree.getroot()

# Funktion zum Entfernen aller Leerzeilen
def remove_empty_lines(element):
    for item in element:
        remove_empty_lines(item)
    if element.tag:
        element.text = element.text.strip() if element.text else None
        element.tail = element.tail.strip() if element.tail else None

# Entferne alle Leerzeilen
remove_empty_lines(root)

# Speichere die bereinigte XML-Datei
tree.write(existing_xml_file, encoding='utf-8', xml_declaration=True)

print(f"Die XML-Datei '{existing_xml_file}' wurde bereinigt und gespeichert.")


Die XML-Datei 'C:\Users\Julius Guizetti\IdeaProjects\matsim-hamburg-HA2\scenarios\input\adapt_transit-schedule_lines.xml' wurde bereinigt und gespeichert.


In [25]:
#Hilfe für formatierung
import xml.dom.minidom as md
# Dateiname der XML-Datei
existing_xml_file = r'C:\Users\Julius Guizetti\IdeaProjects\matsim-hamburg-HA2\scenarios\input\adapt_transit-schedule_lines.xml'
# Lese die XML-Datei ein
dom = md.parse(existing_xml_file)

# Schreibe die XML-Datei mit formatierter Ausgabe
with open(existing_xml_file, 'w', encoding='utf-8') as f:
    dom.writexml(f, addindent='\t', newl='\n')

print(f"Die XML-Datei '{existing_xml_file}' wurde wieder formatiert und gespeichert.")



Die XML-Datei 'C:\Users\Julius Guizetti\IdeaProjects\matsim-hamburg-HA2\scenarios\input\adapt_transit-schedule_lines.xml' wurde wieder formatiert und gespeichert.


In [3]:
# CREATE DEPARTURES
import xml.etree.ElementTree as ET
import pandas as pd

# Hilfsfunktion zur Formatierung der Zeit in Sekunden nach hh:mm:ss
def format_time(seconds):
    hours = seconds // 3600
    minutes = (seconds % 3600) // 60
    seconds = seconds % 60
    return f"{hours:02}:{minutes:02}:{seconds:02}"

# Einlesen der Vehicle IDs aus der transit_vehicles.xml Datei
vehicles_tree = ET.parse(r'C:\Users\Julius Guizetti\IdeaProjects\matsim-hamburg-HA2\scenarios\input\adapt_transit_vehicles.xml')
vehicles_root = vehicles_tree.getroot()

ab_vehicles = [v.get('id') for v in vehicles_root.findall(".//vehicle") if v.get('id').startswith('AB_U5')]
ba_vehicles = [v.get('id') for v in vehicles_root.findall(".//vehicle") if v.get('id').startswith('BA_U5')]

# Einlesen und Parsen der transitSchedule.xml Datei
schedule_tree = ET.parse(r'C:\Users\Julius Guizetti\IdeaProjects\matsim-hamburg-HA2\scenarios\input\adapt_transit-schedule.xml')
schedule_root = schedule_tree.getroot()

# Find the transit routes for AB and BA
u5_ab_route = schedule_root.find(".//transitRoute[@id='U5_AB']")
u5_ba_route = schedule_root.find(".//transitRoute[@id='U5_BA']")

# Erstellen der Departures basierend auf der Taktung
departure_id_counter_ab = 1
departure_id_counter_ba = 1

# Function to create departures based on the taktung
def create_departures(route, vehicles, direction):
    global departure_id_counter_ab, departure_id_counter_ba
    departures = ET.Element('departures')

    taktung = [
        (0, 2, 300), (4, 6, 300), (6, 9, 90), (9, 16, 180),
        (16, 19, 90), (19, 26, 300), (28, 30, 300), (30, 33, 90), (33, 36, 180)
    ]

    for start_hour, end_hour, interval in taktung:
        start_time = start_hour * 3600
        end_time = end_hour * 3600
        current_time = start_time

        while current_time < end_time:
            if direction == 'AB':
                departure_id = f"AB_U5_Dep_{departure_id_counter_ab:03}"
                vehicle_ref = vehicles[(departure_id_counter_ab - 1) % len(vehicles)]
                departure_id_counter_ab += 1
            else:
                departure_id = f"BA_U5_Dep_{departure_id_counter_ba:03}"
                vehicle_ref = vehicles[(departure_id_counter_ba - 1) % len(vehicles)]
                departure_id_counter_ba += 1

            departure = ET.SubElement(departures, 'departure')
            departure.set('id', departure_id)
            departure.set('departureTime', format_time(current_time))
            departure.set('vehicleRefId', vehicle_ref)

            current_time += interval

    route.append(departures)

# Departures für beide Richtungen erstellen
create_departures(u5_ab_route, ab_vehicles, 'AB')
create_departures(u5_ba_route, ba_vehicles, 'BA')

# XML-Dokument formatieren und speichern
def format_xml(element, level=0):
    indent = "    "
    if len(element):
        if not element.text or not element.text.strip():
            element.text = "\n" + indent * (level + 1)
        if not element.tail or not element.tail.strip():
            element.tail = "\n" + indent * level
        for elem in element:
            format_xml(elem, level + 1)
        if not element.tail or not element.tail.strip():
            element.tail = "\n" + indent * level
    else:
        if level and (not element.tail or not element.tail.strip()):
            element.tail = "\n" + indent * level
            
format_xml(schedule_root)

schedule_tree.write(r'C:\Users\Julius Guizetti\IdeaProjects\matsim-hamburg-HA2\scenarios\input\adapt_transitSchedule_with_departures.xml', encoding='utf-8', xml_declaration=True)


In [11]:
# Offsets Berechnung

import pandas as pd
from datetime import timedelta

# Datei einlesen (Passe den Dateipfad entsprechend an)
file_path = 'stationen_ab.csv'
df = pd.read_csv(file_path, sep=';')


# Geschwindigkeit in m/s
speed_m_per_s = 11

# Initiale Zeit für die erste Station
initial_time = timedelta(hours=0, minutes=0, seconds=0)
departure_interval = timedelta(seconds=30)

# Offsets berechnen
arrival_offsets = []
departure_offsets = []

for i, row in df.iterrows():
    if i == 0:
        # Für die erste Station
        arrival_offset = initial_time
        departure_offset = arrival_offset + departure_interval
    else:
        # Für nachfolgende Stationen
        previous_departure_offset = departure_offsets[-1]
        if pd.notna(row['distanceToNextNode']):
            travel_time = timedelta(seconds=row['distanceToNextNode'] / speed_m_per_s)
        else:
            # Standardwert setzen, wenn distanceToNextNode NaN ist, z.B. 0 Sekunden
            travel_time = timedelta(seconds=0)
        arrival_offset = previous_departure_offset + travel_time
        departure_offset = arrival_offset + departure_interval
    
    arrival_offsets.append(arrival_offset)
    departure_offsets.append(departure_offset)

# Formatieren der Offsets in HH:MM:SS
df['arrivalOffset'] = [str(offset).split('.')[0] for offset in arrival_offsets]
df['departureOffset'] = [str(offset).split('.')[0] for offset in departure_offsets]

# Datei mit neuen Offsets speichern (Passe den Ausgabe-Dateipfad entsprechend an)
output_path = 'stationen_ab.csv'
df.to_csv(output_path, sep=';')

